# OpenAI Function Calling


In [1]:
import os
import openai
from utils import format_message, show_prompt

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""

    # Dummy response
    # Chama API
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }

    return json.dumps(weather_info)

In [3]:
# define a function
# https://platform.openai.com/docs/guides/function-calling

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [5]:
import openai

llm_model = "gpt-4o-mini"
client = openai.OpenAI()


In [6]:
# Call the responses endpoint

response = client.chat.completions.create(
        model=llm_model,
        messages=messages,
        tools=tools,
        temperature=0,
    )

In [7]:
print(response)

ChatCompletion(id='chatcmpl-CVPy8eWH3BMvSutUDICE6eunEufD0', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_wDhlGOZSn2WgAcS92unM27AY', function=Function(arguments='{"location":"Boston, MA"}', name='get_current_weather'), type='function')]))], created=1761604028, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=17, prompt_tokens=79, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [8]:
response_message = response.choices[0].message

In [9]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_wDhlGOZSn2WgAcS92unM27AY', function=Function(arguments='{"location":"Boston, MA"}', name='get_current_weather'), type='function')])

In [12]:
print(response_message.content)

None


In [13]:
print(response_message.tool_calls)

[ChatCompletionMessageFunctionToolCall(id='call_wDhlGOZSn2WgAcS92unM27AY', function=Function(arguments='{"location":"Boston, MA"}', name='get_current_weather'), type='function')]


In [ ]:
#for tool_call in response_message.tool_calls:
#    print("Tool Call Name:", tool_call.function.name)
#    print("Tool Call Arguments:", json.loads(tool_call.function.arguments))

print("Function Name:", response_message.tool_calls[0].function.name)
print("Function Arguments:", json.loads(response_message.tool_calls[0].function.arguments))

Function Name: get_current_weather
Function Arguments: {'location': 'Boston, MA'}


In [15]:
args = json.loads(response_message.tool_calls[0].function.arguments)
args

{'location': 'Boston, MA'}

In [16]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

### LangChain Tools

In [17]:
from langchain.tools import tool
import requests

In [ ]:
@tool
def get_weather(city: str) -> str:
    """
    Obtém a previsão do tempo para a cidade informada.
    Args:
        city: Nome da cidade (ex: 'Recife, PE, Brasil')
    Returns:
        String com resumo da previsão do tempo.
    """
    # Aqui uma chamada fictícia; substitua pela real:
    # response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid=SEU_API_KEY&units=metric")
    # data = response.json()
    # resumo = f"A temperatura em {city} agora é {data['main']['temp']}°C com {data['weather'][0]['description']}."
    resumo = f"[Serviço fictício] A previsão para {city} é: 26°C, céu parcialmente nublado."
    return resumo


def thinking_tool(reflexion:str):
    """Chame a ferramenta de reflexão para analisar a resposta."""
    return f"[Ferramenta de reflexão] Analisando: {reflexion}"




In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=llm_model,
    temperature=0.7
    )

# Vincula a tool ao modelo
llm_with_tools = llm.bind_tools([get_weather])

In [23]:
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
)


user_input = "Ola"

result = llm_with_tools.invoke([HumanMessage(content=user_input)])


# Verifica se houve chamado da tool
if hasattr(result, "tool_calls") and result.tool_calls:

    tool_call = result.tool_calls[0]
    tool_name = tool_call["name"]
    args = tool_call["args"]


    if tool_name != "get_weather":
        raise ValueError(f"Tool inesperada: {tool_name}")

    # Executa a tool
    tool_result = get_weather.invoke(args)

    tool_message = ToolMessage(
        name=tool_name,
        content=str(tool_result),
        tool_call_id=tool_call["id"]
    )


    # Envia de volta ao modelo o resultado da tool, para que ele complete a resposta
    followup = llm_with_tools.invoke([
        HumanMessage(content=user_input),
        result,
        tool_message
    ])

    print(followup.content)

else:
    print(result.content)

Olá! Como posso ajudá-lo hoje?


In [22]:
result.tool_calls[0]

{'name': 'get_weather',
 'args': {'city': 'Recife, PE, Brasil'},
 'id': 'call_N2vhOmGJb8UmcwEf1icMuCOp',
 'type': 'tool_call'}